In [0]:
def DataLoad(ID,name):
  # Code to read csv file into colaboratory:
  !pip install -U -q PyDrive
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials

  # 1. Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  #2. Get the file
  downloaded = drive.CreateFile({'id':ID}) # replace the id with id of file you want to access
  downloaded.GetContentFile(name)  

  #3. Read file as panda dataframe
  import pandas as pd
  DATA = pd.read_csv(name)
  DATA = DATA.values
  return DATA

In [8]:
!pip install -q keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization
from keras.optimizers import SGD
from keras import regularizers, optimizers
import numpy as np
import tensorflow as tf

## Load Data
Obama1_ID_X = '1j8xblrjXOuAy2S_v4Qlo_hPJ4BWSUME5'
Obama1_name_X = 'obama1_X.csv'
Obama1_ID_Y = '1e8LfCcNRDJkzOaQKBO9HHNqU4DV_DPCh'
Obama1_name_Y = 'obama1_Y.csv'
Trump1_ID_X ='1Xtp9-M76UjcHnqTYpeawUM6zLHznrp-B'
Trump1_name_X = 'trump1_X.csv'
Trump1_ID_Y ='1mVVuBg3HKsJ2qX-nu_sq8bAjNZhsxUfm'
Trump1_name_Y = 'trump1_Y.csv'
trainPercent = 0.95 # % of samples to train on

Obama1_X_DATASET = DataLoad(Obama1_ID_X,Obama1_name_X)
Obama1_Y_DATASET = DataLoad(Obama1_ID_Y,Obama1_name_Y)
Trump1_X_DATASET = DataLoad(Trump1_ID_X,Trump1_name_X)
Trump1_Y_DATASET = DataLoad(Trump1_ID_Y,Trump1_name_Y)
X_DATASET=np.vstack((Obama1_X_DATASET,Trump1_X_DATASET))
Y_DATASET=np.vstack((Obama1_Y_DATASET,Trump1_Y_DATASET))
DATASET = np.hstack((X_DATASET,Y_DATASET))
np.random.shuffle(DATASET)
X_DATASET = DATASET[:,0:X_DATASET.shape[1]]
Y_DATASET = DATASET[:,X_DATASET.shape[1]:DATASET.shape[1]]

mtrain = int(trainPercent*X_DATASET.shape[0])
x_train = X_DATASET[0:mtrain,0:X_DATASET.shape[1]]
x_test  = X_DATASET[mtrain:X_DATASET.shape[0],0:X_DATASET.shape[1]]
y_train = Y_DATASET[0:mtrain,0:Y_DATASET.shape[1]]
y_test  = Y_DATASET[mtrain:Y_DATASET.shape[0],0:Y_DATASET.shape[1]]

classes = y_train.shape[1]  # num speakers
n_vec =   x_train.shape[1]  # length of feature vector
m_train = x_train.shape[0]  # num training examples
m_test =  x_test.shape[0]   # num test examples
params = {}                 # initalize params

print ("number of training examples = " + str(m_train))
print ("number of test examples = " + str(m_test))
print("number of classes: " + str(classes))
print ("X_train shape: " + str(x_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(x_test.shape))
print ("Y_test shape: " + str(y_test.shape))

#################### Hyperparameters ####################
L = 5                                   # num layers in network
bsize = 32                              # minibatch size
epochnum = 20                           # num epochs for optimization
learning_rate = 0.001                   # params for Adam
learning_decay = 0.00001                # learning decay for Adam
BN_mom = 0.99                           # momentum for batch norm
L2 = np.array([.1,.1,.1,.1,.1])         # L2 lambda parameters for each layer
N = np.array([390,390,390,390,classes]) # num nodes/layer for each hidden layer
#########################################################


for l in range(1,L+1):
    params["L2_"+str(l)] = L2[l-1]
    params["N"+str(l)] = N[l-1]

## NN Model (X->RELU->RELU->RELU->RELU->SOFTMAX)
model = Sequential()
model.add(Dense(params['N1'], activation='relu', input_dim=n_vec,use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=regularizers.l2(params["L2_1"]), bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None,name='fc1'))
model.add(BatchNormalization(axis=1, momentum=BN_mom, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones',name='bn1'))
for l in range(2,L-1):
    model.add(Dense(params['N'+str(l)], activation='relu',use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=regularizers.l2(params["L2_" + str(l)]), bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None,name='fc'+str(l)))
    model.add(BatchNormalization(axis=1, momentum=BN_mom, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones',name='bn'+str(l)))
model.add(Dense(params['N'+str(L)], activation='softmax',use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=regularizers.l2(params["L2_" + str(L)]), bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None,name='fc'+str(L)))

## Compile and Train Model
adam_opt = optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=learning_decay, amsgrad=False)
model.compile(loss='categorical_crossentropy',optimizer=adam_opt,metrics=['accuracy'])
model.fit(x_train, y_train,epochs=epochnum,batch_size=bsize)

## Test Model
preds = model.evaluate(x_test, y_test, batch_size=bsize)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

## Model Predictions of Test Set
y_hat = model.predict(x_test, batch_size=bsize)

## Display Model Architecture
model.summary()


number of training examples = 41466
number of test examples = 2183
number of classes: 2
X_train shape: (41466, 13)
Y_train shape: (41466, 2)
X_test shape: (2183, 13)
Y_test shape: (2183, 2)
Epoch 1/20
41466/41466 [==============================] - 16s 377us/step - loss: 2.8671 - acc: 0.8519
Epoch 2/20
 8960/41466 [=====>........................] - ETA: 11s - loss: 0.4530 - acc: 0.8632

41466/41466 [==============================] - 15s 364us/step - loss: 0.4446 - acc: 0.8630
Epoch 3/20
41466/41466 [==============================] - 15s 363us/step - loss: 0.4052 - acc: 0.8673
Epoch 4/20
17472/41466 [===========>..................] - ETA: 8s - loss: 0.3776 - acc: 0.8723

41466/41466 [==============================] - 15s 361us/step - loss: 0.3698 - acc: 0.8715
Epoch 5/20
41466/41466 [==============================] - 15s 363us/step - loss: 0.3502 - acc: 0.8746
Epoch 6/20
20032/41466 [=============>................] - ETA: 7s - loss: 0.3390 - acc: 0.8779

41466/41466 [==============================] - 15s 361us/step - loss: 0.3370 - acc: 0.8771
Epoch 7/20
41466/41466 [==============================] - 15s 362us/step - loss: 0.3330 - acc: 0.8776
Epoch 8/20
20928/41466 [==============>...............] - ETA: 7s - loss: 0.3330 - acc: 0.8781

41466/41466 [==============================] - 15s 357us/step - loss: 0.3283 - acc: 0.8785
Epoch 9/20
41466/41466 [==============================] - 15s 361us/step - loss: 0.3235 - acc: 0.8812
Epoch 10/20
20928/41466 [==============>...............] - ETA: 7s - loss: 0.3239 - acc: 0.8796

...

41466/41466 [==============================] - 15s 359us/step - loss: 0.3275 - acc: 0.8774
Epoch 13/20
41466/41466 [==============================] - 15s 360us/step - loss: 0.3209 - acc: 0.8804
Epoch 14/20
20960/41466 [==============>...............] - ETA: 7s - loss: 0.3182 - acc: 0.8802

41466/41466 [==============================] - 15s 359us/step - loss: 0.3199 - acc: 0.8786
Epoch 15/20
41466/41466 [==============================] - 15s 356us/step - loss: 0.3187 - acc: 0.8806
Epoch 16/20
20960/41466 [==============>...............] - ETA: 7s - loss: 0.3221 - acc: 0.8782

41466/41466 [==============================] - 15s 356us/step - loss: 0.3175 - acc: 0.8815
Epoch 17/20
41466/41466 [==============================] - 15s 358us/step - loss: 0.3159 - acc: 0.8812
Epoch 18/20
20928/41466 [==============>...............] - ETA: 7s - loss: 0.3191 - acc: 0.8761

41466/41466 [==============================] - 15s 356us/step - loss: 0.3163 - acc: 0.8790
Epoch 19/20
41466/41466 [==============================] - 15s 356us/step - loss: 0.3137 - acc: 0.8819
Epoch 20/20
21184/41466 [==============>...............] - ETA: 7s - loss: 0.3160 - acc: 0.8782

2183/2183 [==============================] - 0s 128us/step
Loss = 0.30753354559619656
Test Accuracy = 0.8818140174345418
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 390)               5460      
_________________________________________________________________
bn1 (BatchNormalization)     (None, 390)               1560      
_________________________________________________________________
fc2 (Dense)                  (None, 390)               152490    
_________________________________________________________________
bn2 (BatchNormalization)     (None, 390)               1560      
_________________________________________________________________
fc3 (Dense)                  (None, 390)               152490    
_________________________________________________________________
bn3 (BatchNormalization)     (None, 390)               1560      
_____________________